In [31]:
import pandas as pd
import time
from tqdm import tqdm_notebook

In [43]:
comp=pd.read_excel('./project_data/comp_full_token_ver6_210219')

In [44]:
comp.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', '내역사업명', 'title', '연구목표', '연구내용', '기대효과',
       '사업명', '부처명', '지역구분', '6T관련기술분류', '국가전략기술', '한글키워드', '영문키워드',
       '과제관리(전문)기관명', '연구', '연구_okt', '연구_hannanum', '연구_krwordrank',
       '연구_kkma'],
      dtype='object')

In [45]:
del comp['Unnamed: 0']

In [46]:
del comp['Unnamed: 0.1']

In [47]:
data=comp

In [48]:
data.columns

Index(['내역사업명', 'title', '연구목표', '연구내용', '기대효과', '사업명', '부처명', '지역구분',
       '6T관련기술분류', '국가전략기술', '한글키워드', '영문키워드', '과제관리(전문)기관명', '연구', '연구_okt',
       '연구_hannanum', '연구_krwordrank', '연구_kkma'],
      dtype='object')

In [49]:
#불용어 제거

'''============불용어 stopword 정의============'''
stopword=''
f =open("./project_data/stop_words.txt","r", encoding = 'utf-8')
for line in f:
    stopword += line

# krwordrank에 hannanum 돌리기

In [50]:
from ckonlpy.tag import Twitter
twitter = Twitter()

def okt_tokenize(text):
    okt_nouns=[]
    
    text_pos_list = twitter.pos(text)
    for word, tag in text_pos_list:
        if (tag in ('Alpha', "Noun")) and (not word in stopword):
            if len(word)>1:
                okt_nouns.append(word)

    okt_nouns=list(set(okt_nouns))
    
    okt_nouns=' '.join(okt_nouns)

    return okt_nouns

C:\Users\user\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [51]:
twitter.add_dictionary(['머신러닝','딥러닝','인공지능','빅데이터','machine Learning','machineLearning',
                        'deep learning','bigdata','의과학','의공학','데이터','라이센서','핸드피스',
                        'hand piece','handpiece','sw','하이브리드','보철물','지르코니아','성형체','우회기법',
                        '고반응성','3d','o2o','5g','4세대','2d','5세대','co2','4k','2차원','3차원'], 'Noun')


In [52]:
start_time = time.time()
data['krwr_okt']=''

i=0
for row in tqdm_notebook(data['연구_krwordrank']): 
    data['krwr_okt'].iloc[i]=okt_tokenize(row)
    i+=1
    
end_time = time.time()
print("실행시간 : ", end_time-start_time)
    

<ipython-input-52-2068862e00c7>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(data['연구_krwordrank']):



실행시간 :  51.93623495101929


# krwordrank에 hannanum 돌리기

In [53]:
from konlpy.tag import Hannanum

hannanum = Hannanum()

def hannanum_tokenize(text):
    han_nouns=[]
    text_pos_list = hannanum.pos(text)
    for word, tag in text_pos_list:
        if (tag in ('F', "N")) and (not word in stopword):
            if len(word)>1:
                han_nouns.append(word)

    han_nouns=list(set(han_nouns))
    
    han_nouns=' '.join(han_nouns)

    return han_nouns

In [54]:
start_time = time.time()
data['krwr_han']=''

i=0
try:
    for row in tqdm_notebook(data['연구_krwordrank']): 
        data['krwr_han'].iloc[i]=hannanum_tokenize(row)
        i+=1

    end_time = time.time()
    print("실행시간 : ", end_time-start_time)
except:
    print(i)
    pass
    

<ipython-input-54-2ac2c0b6543f>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for row in tqdm_notebook(data['연구_krwordrank']):



실행시간 :  100.76774382591248


In [56]:
#엑셀파일 저장
writer = pd.ExcelWriter('./project_data/comp_ver7_210219.xlsx', engine='xlsxwriter')  #경로 각자에 맞게 변경
data.to_excel(writer)
writer.save()